In [ ]:
!pip install langchain==0.0.232 faiss-cpu streamlit pyngrok


In [ ]:
sample_text = """
Python is a programming language that lets you work quickly
and integrate systems more effectively.

LangChain helps build applications with LLMs by chaining
prompts, agents, and memory.

Ollama provides access to LLAMA2 models locally.
"""

with open("sample_docs.txt", "w") as f:
    f.write(sample_text)


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load document
loader = TextLoader("sample_docs.txt")
documents = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

print("Number of document chunks:", len(docs))


Number of document chunks: 1


In [ ]:
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import FAISS
import numpy as np

# Mock embeddings (random vectors)
class MockEmbeddings(Embeddings):
    def embed_documents(self, texts):
        return [np.random.rand(1536).tolist() for _ in texts]
    def embed_query(self, text):
        return np.random.rand(1536).tolist()

embeddings = MockEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()


In [ ]:
# Simple function to simulate RAG retrieval and response
def rag_chatbot(query):
    # Retrieve docs (top 2 chunks)
    results = retriever.get_relevant_documents(query)[:2]
    context_text = "\n".join([doc.page_content for doc in results])

    # Simulated answer
    answer = f"Answer based on context:\n{context_text}\n\nYour question: {query}"
    return answer


In [ ]:
question = "What is LangChain?"
response = rag_chatbot(question)
print(response)


Answer based on context:
Python is a programming language that lets you work quickly
and integrate systems more effectively.

LangChain helps build applications with LLMs by chaining
prompts, agents, and memory.

Ollama provides access to LLAMA2 models locally.

Your question: What is LangChain?


In [ ]:
%%writefile app.py
import streamlit as st

st.title("RAG Chatbot Demo (Colab)")

question = st.text_input("Ask a question:")

if question:
    # Simple mock RAG answer
    answer = f"Answer based on context:\nPython is a programming language...\nLangChain helps build applications...\nYour question: {question}"
    st.write(answer)


Writing app.py


In [ ]:
# Ask multiple questions
while True:
    question = input("Ask a question (or type 'exit' to stop): ")
    if question.lower() == "exit":
        break
    print(rag_chatbot(question))



Ask a question (or type 'exit' to stop): what is python?
Answer based on context:
Python is a programming language that lets you work quickly
and integrate systems more effectively.

LangChain helps build applications with LLMs by chaining
prompts, agents, and memory.

Ollama provides access to LLAMA2 models locally.

Your question: what is python?
Ask a question (or type 'exit' to stop): exit
